In [7]:
import pandas as pd

df = pd.read_excel("/dataset.xlsx")
df.head()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.916667,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.916667,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583333,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500000,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833333,5.0,390.56840,5,24.97937,121.54245,43.1


In [8]:
# Converte la variabile target da "10.000 NT$/Ping" a "NT$/m²"
df2 = df.copy()
df2["Y house price per m2"] = df["Y house price of unit area"] * 10000 / 3.3
del df2["Y house price of unit area"]
df2.head()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price per m2
0,1,2012.916667,32.0,84.87882,10,24.98298,121.54024,114848.484848
1,2,2012.916667,19.5,306.59470,9,24.98034,121.53951,127878.787879
2,3,2013.583333,13.3,561.98450,5,24.98746,121.54391,143333.333333
3,4,2013.500000,13.3,561.98450,5,24.98746,121.54391,166060.606061
4,5,2012.833333,5.0,390.56840,5,24.97937,121.54245,130606.060606


In [9]:

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X = df2[["X5 latitude", "X6 longitude"]]
y = df2["Y house price per m2"]

model = LinearRegression()
model.fit(X, y)


LinearRegression()

In [10]:
!pip install statsmodels




In [11]:
import statsmodels.api as sm

# Aggiungiamo una costante (intercetta)
X_with_const = sm.add_constant(X)

# Stima del modello OLS (Ordinary Least Squares)
model_ols = sm.OLS(y, X_with_const).fit()

# Mostra il summary
print(model_ols.summary())

                             OLS Regression Results                             
Dep. Variable:     Y house price per m2   R-squared:                       0.405
Model:                              OLS   Adj. R-squared:                  0.402
Method:                   Least Squares   F-statistic:                     140.1
Date:                  Thu, 03 Apr 2025   Prob (F-statistic):           4.16e-47
Time:                          13:23:10   Log-Likelihood:                -4878.9
No. Observations:                   414   AIC:                             9764.
Df Residuals:                       411   BIC:                             9776.
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.501e+08   1.

In [12]:
!pip install pygam


ERROR: Operation cancelled by user


In [2]:
!pip install streamlit
!pip install joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.2 MB/s eta 0:00:00


In [13]:
import joblib

joblib.dump(model_ols, "model_ols.joblib")


['model_ols.joblib']

In [14]:
import streamlit as st
import pandas as pd
import statsmodels.api as sm
import joblib

# Caricamento del modello e dei limiti geografici
model_ols = joblib.load("model_ols.joblib")  # Salva prima il modello con joblib!
lat_min, lat_max = 24.93, 25.08  # <-- aggiorna con i tuoi valori reali
lon_min, lon_max = 121.47, 121.57

st.title("Stima del Prezzo Immobiliare (per m²) - Sindian, Taiwan")

latitude = st.number_input("Inserisci la latitudine", value=25.0)
longitude = st.number_input("Inserisci la longitudine", value=121.5)

if latitude < lat_min or latitude > lat_max:
    st.error(f"Latitudine fuori dai limiti del dataset! ({lat_min} - {lat_max})")
elif longitude < lon_min or longitude > lon_max:
    st.error(f"Longitudine fuori dai limiti del dataset! ({lon_min} - {lon_max})")
else:
    # Costruzione DataFrame per predizione
    input_data = pd.DataFrame([{
        "const": 1,
        "X5 latitude": latitude,
        "X6 longitude": longitude
    }])[["const", "X5 latitude", "X6 longitude"]]

    prediction = model_ols.predict(input_data)[0]
    st.success(f"Prezzo stimato: {prediction:,.2f} NT$/m²")


2025-04-03 13:23:31.762 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-03 13:23:32.078 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-04-03 13:23:32.079 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-03 13:23:32.083 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-03 13:23:32.086 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-03 13:23:32.087 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-03 13:23:32.091 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-03 13:23:32.092 Session state does not 

In [17]:
from google.colab import files
files.download("model_ols.joblib")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>